# Setup

In [1]:
import os
import json
from langchain_openai import ChatOpenAI,OpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

with open('../.env', 'r') as config_file:
    config = json.loads(config_file.read())

os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']
os.environ['OPENAI_BASE_URL'] = config['OPENAI_BASE_URL']
os.environ['OPENAI_API_BASE'] = config['OPENAI_BASE_URL']
os.environ['DEFAULT_MODEL'] = config['DEFAULT_MODEL']
os.environ['GOOGLE_API_KEY'] = config['GOOGLE_API_KEY']
model = os.environ['DEFAULT_MODEL']

# Completion

In [2]:
template = '''
Você é um analista financeiro.
Escreva um relatório financeiro detalhado para a empresa "{empresa}" para o período {periodo}.

O relatório deve ser escrito em {idioma} e incluir as seguintes análises:
{analises}

Certifique-se de fornecer insights e conclusões para cada seção.
'''

In [3]:
prompt_template = PromptTemplate.from_template(template=template)

In [4]:
empresa = 'ACME Corp'
periodo = 'Q1 2024'
idioma = 'Português'
analises = [
    "Análise do Balanço Patrimonial",
    "Análise do Fluxo de Caixa",
    "Análise de Tendências",
    "Análise de Receita e Lucro",
    "Análise de Posição de Mercado"
]
analises_text = "\n".join([f"- {analise}" for analise in analises])

In [5]:
print(analises_text)

- Análise do Balanço Patrimonial
- Análise do Fluxo de Caixa
- Análise de Tendências
- Análise de Receita e Lucro
- Análise de Posição de Mercado


In [6]:
prompt = prompt_template.format(
    empresa=empresa,
    periodo=periodo,
    idioma=idioma,
    analises=analises_text
)
print("Prompt Gerado:\n", prompt)

Prompt Gerado:
 
Você é um analista financeiro.
Escreva um relatório financeiro detalhado para a empresa "ACME Corp" para o período Q1 2024.

O relatório deve ser escrito em Português e incluir as seguintes análises:
- Análise do Balanço Patrimonial
- Análise do Fluxo de Caixa
- Análise de Tendências
- Análise de Receita e Lucro
- Análise de Posição de Mercado

Certifique-se de fornecer insights e conclusões para cada seção.



In [7]:
openai = OpenAI(model_name=model, max_tokens=2000) #max 4096

response = openai.invoke(prompt)
print("Saída do LLM:\n", response)

Saída do LLM:
 <|channel|>analysis<|message|>We need to produce a financial report in Portuguese, detailed. Provide sections with analysis and conclusions. No data given; we can assume hypothetical or generic. We'll create plausible numbers. Ensure each section has insights. Let's output.<|end|><|start|>assistant<|channel|>final<|message|>**Relatório Financeiro – ACME Corp – Q1 2024**

| Indicador | Valor (R$) | Variação YoY |
|-----------|------------|--------------|
| Receita Líquida | 12 500 000,00 | +8,3 % |
| Lucro Líquido | 2 300 000,00 | +15,5 % |
| EBITDA | 4 100 000,00 | +10,1 % |
| Ativos Totais | 35 200 000,00 | –1,2 % |
| Passivo Circulante | 12 800 000,00 | +3,0 % |
| Patrimônio Líquido | 19 400 000,00 | +7,8 % |

> **Observação:** Os valores são fictícios e servem apenas como base para a demonstração de análises.

---

## 1. Análise do Balanço Patrimonial

### 1.1 Ativo
- **Ativos Circulantes** (R$ 10 600 000,00) representaram 30% dos ativos totais, um aumento moderado em

# Chat

In [8]:
#equivalência aos roles: system: system, Human: user, AI: assistant
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='Você deve estruturar suas respostas de acordo com o método de análise de negócios, garantindo clareza e concisão.'),
        HumanMessagePromptTemplate.from_template('Por favor, gere um relatório detalhado sobre a indústria de tecnologia na região "{regiao}".'),
        AIMessage(content='Claro, vou começar coletando informações sobre a região e analisando os dados disponíveis.'),
        HumanMessage(content='Certifique-se de incluir uma análise SWOT e uma previsão de crescimento para os próximos 5 anos.'),
        AIMessage(content='Entendido. Aqui está o relatório completo:')
    ]
)

In [9]:
prompt_gerado = chat_template.format_messages(regiao='América Latina')
print(prompt_gerado)  

[SystemMessage(content='Você deve estruturar suas respostas de acordo com o método de análise de negócios, garantindo clareza e concisão.'), HumanMessage(content='Por favor, gere um relatório detalhado sobre a indústria de tecnologia na região "América Latina".'), AIMessage(content='Claro, vou começar coletando informações sobre a região e analisando os dados disponíveis.'), HumanMessage(content='Certifique-se de incluir uma análise SWOT e uma previsão de crescimento para os próximos 5 anos.'), AIMessage(content='Entendido. Aqui está o relatório completo:')]


In [10]:
openai = ChatOpenAI(model_name='gpt-3.5-turbo')
response = openai.invoke(prompt_gerado)
print("Saída do LLM:\n", response.content)

Saída do LLM:
 **Relatório de Análise da Indústria de Tecnologia na América Latina  
(Período: 2024 – 2030)**  

| Seção | Conteúdo |
|-------|----------|
| 1. Visão geral do mercado | Tamanho atual, principais players, segmentos (hardware, software, serviços, fintechs, health‑tech, edtech) |
| 2. Ambiente regulatório e macroeconômico | Políticas de TI, incentivos fiscais, estabilidade cambial, PIB, taxa de desemprego |
| 3. Análise SWOT | Forças, Fraquezas, Oportunidades, Ameaças (com exemplos concretos) |
| 4. Tendências tecnológicas | IA generativa, edge computing, 5G/6G, cloud‑native, cybersecurity, blockchain |
| 5. Projeção de crescimento | CAGR estimado, cenários base/otimista/pessimista, principais impulsores e gargalos |
| 6. Recomendações estratégicas | Entrada em mercado, parcerias, foco em nichos, investimento em talento local |

---

## 1. Visão Geral do Mercado

| País (top 5 por PIB) | Receita de TI (US$ bilhões) | Crescimento CAGR 2020‑24* |
|----------------------|----